In [ ]:
!ls

sample_data


#External Databases

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-zvftc10h
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-zvftc10h
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5 MB 9.0 MB/s 
     |████████████████████████████████| 67 kB 7.1 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 895 kB 59.7 MB/s 
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3835851 sha256=5c6f3f6b6eb96885e38fc39df988e2637dc6c916331038a16c84693ad744954c
  Stored in directory: /tmp/pip-ephem-wheel-cache-oq3q0ya2/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [ ]:
! pip install ray[tune]

     |████████████████████████████████| 52.7 MB 60 kB/s 
     |████████████████████████████████| 175 kB 12.2 MB/s 
     |████████████████████████████████| 4.1 MB 41.4 MB/s 
     |████████████████████████████████| 125 kB 52.3 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [ ]:
!pip install datasets

     |████████████████████████████████| 312 kB 9.5 MB/s 
     |████████████████████████████████| 134 kB 50.4 MB/s 
     |████████████████████████████████| 1.1 MB 26.9 MB/s 
     |████████████████████████████████| 212 kB 53.8 MB/s 
     |████████████████████████████████| 127 kB 51.2 MB/s 
     |████████████████████████████████| 271 kB 50.9 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 144 kB 49.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4385ff5c7284c18d98381824f968d5ed6faaaaead0d8c271731c897a70ab797c
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


#Libraries

In [ ]:
from google.colab import files

import os
import re
import json
import string

import torch
import torch.nn as nn

import numpy as np
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers

from datasets import load_metric
from sklearn.metrics import f1_score, precision_score, recall_score
from seqeval.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as prfs

import tensorflow_hub as hub
from keras import backend as K

import transformers
print(transformers.__version__)
from transformers import TrainingArguments
from transformers import Trainer
from transformers.trainer_utils import EvalLoopOutput

from transformers import AutoTokenizer, AutoModel
BertTokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
BertEmbModel = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

4.18.0.dev0


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Loading Dataset

In [ ]:
uploaded = files.upload()

Saving test_500_v2.conll to test_500_v2.conll
Saving train_1500_v2.conll to train_1500_v2.conll


In [ ]:
class TDMSciDataset(torch.utils.data.Dataset):
  def __init__(self, raw_x, raw_y, max_length=250):
    self.raw_x = raw_x
    self.raw_y = raw_y
    
    self.max_length = max_length

  def tokenize_and_preserve_labels(self, sentence, text_labels):
    """
    The tokenizer can split single words into multiple tokens - this breaks
    the labels, so we need to keep track of this!
    """
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = BertTokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

  def __getitem__(self, idx):
    tokens = self.raw_x[idx]
    labels = np.zeros((len(tokens)))
    for i, label in enumerate(self.raw_y[idx]):
      labels[i] = label

    # This could be moved to __init__ to save time?
    tokens, labels = self.tokenize_and_preserve_labels(tokens, labels)

    # Convert each token to an id number 
    input_ids = BertTokenizer.convert_tokens_to_ids(tokens)
    
    # add and adjust for special tokens
    input_ids = [BertTokenizer.cls_token_id] + input_ids + [BertTokenizer.sep_token_id]  
    labels = [0] + labels + [0]

    # Pad inputs
    input_ids = torch.tensor(np.pad(input_ids, [0, self.max_length-len(input_ids)]))
    labels = torch.tensor(np.pad(labels, [0, self.max_length-len(labels)], constant_values=-100))

    attention_mask = torch.tensor([int(i != 0) for i in input_ids])

    #return {input_ids, labels}
    #return {'input_ids': [input_ids], 'labels': labels}
    return {'input_ids': [input_ids], 'attention_mask': [attention_mask], 'labels': labels}

  def __len__(self):
    return len(self.raw_y)


In [ ]:
# Entity maps
def Entity2ID(Sc_Entity):
  return {
        'B-TASK': 1,
        'I-TASK': 2,
        'B-DATASET': 3,
        'I-DATASET': 4,
        'B-METRIC': 5,
        'I-METRIC': 6,
        'None': 0,
        'O': 0,
    }[Sc_Entity]

def ID2Entity(Sc_Entity):
  return {
        1: 'B-TASK',
        2: 'I-TASK',
        3: 'B-DATASET',
        4: 'I-DATASET',
        5: 'B-METRIC',
        6: 'I-METRIC',
        0: 'None',
    }[Sc_Entity]

In [ ]:
# Load raw data train
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('train_1500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1]))  # use this for BIO tags
      # cur_y.append(Entity2ID(line[-1].split('-')[-1]))  # use this for span tags

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
train = TDMSciDataset(raw_x, raw_y)

#print(train[0])

['The', 'results', 'show', 'that', 'proposed', 'shallow', 'representations', 'of', 'sentence', 'structure', 'are', 'robust', 'to', 'reductions', 'in', 'parsing', 'accuracy', ',', 'and', 'that', 'the', 'contribution', 'of', 'alternative', 'representations', 'of', 'sentence', 'structure', 'to', 'successful', 'semantic', 'role', 'labeling', 'varies', 'with', 'the', 'integrity', 'of', 'the', 'parsing', 'and', 'argument', '-', 'identification', 'stages', '.', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0]


In [ ]:
# Load raw data test
raw_x = []
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('test_500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1]))  # use this for BIO tags
      # cur_y.append(Entity2ID(line[-1].split('-')[-1]))  # use this for span tags

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
test = TDMSciDataset(raw_x, raw_y)

#print(test[0])

['We', 'evaluate', 'our', 'approach', 'on', 'three', 'datasets', ':', 'TriviaQA', 'unfiltered', '(', ',', 'a', 'dataset', 'of', 'questions', 'from', 'trivia', 'databases', 'paired', 'with', 'documents', 'found', 'by', 'completing', 'a', 'web', 'search', 'of', 'the', 'questions', ';', 'TriviaQA', 'web', ',', 'a', 'dataset', 'derived', 'from', 'TriviaQA', 'unfiltered', 'by', 'treating', 'each', 'question', 'document', 'pair', 'where', 'the', 'document', 'contains', 'the', 'question', 'answer', 'as', 'an', 'individual', 'training', 'point', ';', 'and', 'SQuAD', '(', ',', 'a', 'collection', 'of', 'Wikipedia', 'articles', 'and', 'crowdsourced', 'questions', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(test), len(train)

(486, 1522)

In [ ]:
#train[1]

In [ ]:
local_hp_train = []
local_hp_val = []
for x in range(500):
  local_hp_train.append(train[x])

for x in range(180):
  local_hp_val.append(test[x])

#print(len(local_hp_train))
#print(local_hp_train[1])

# Pytorch Model Definition

In [ ]:
class NerModel(nn.Module):
  def __init__(self, b_embeddings, emb_dims=768, ff_dims=14, out_dims=7):
    super(NerModel, self).__init__()
    self.sci_embeddings = b_embeddings
    self.embd_dropout = nn.Dropout(0.1)
    self.ff_dropout = nn.Dropout(0.1)
    self.ff = nn.Linear(emb_dims, ff_dims)
    self.tanh = nn.Tanh()
    self.lstm = nn.LSTM(768, 100, 1, bidirectional=True)
    self.lstm_drop = nn.Dropout(0.4)
    self.ff = nn.Linear(200, 14)
    self.ff_act = nn.ReLU()
    self.classifier = nn.Linear(ff_dims, out_dims)
  def forward(self, **inputs):
    embds = self.sci_embeddings(**inputs)['last_hidden_state']
    out = self.embd_dropout(embds)
    out, _ = self.lstm(out)
    out = self.tanh(out)
    out = self.lstm_drop(out)
    out = self.ff(out)
    out = self.ff_act(out)
    out = self.ff_dropout(out)
    out = self.classifier(out)
    return out


# Metrics and Configs

In [ ]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

import math

def load_param():
  for n, v in best_run.hyperparameters.items():
    if n == 'seed':
      setattr(trainer.args, n, math.ceil(v))
      print(n, math.ceil(v))
    else:
      setattr(trainer.args, n, v)
      print(n, v)   

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

batch_size = 4
training_args = TrainingArguments(
    "trained_scibert_ner_model", # output dir
    learning_rate=1e-5, 
    num_train_epochs=10, 
    dataloader_drop_last=True,
    per_device_eval_batch_size=batch_size, 
    per_device_train_batch_size=batch_size,
    save_steps=len(train) // batch_size,
    eval_steps=len(train) // batch_size,
    lr_scheduler_type='cosine',
    evaluation_strategy='steps'
    )
#print(training_args)

In [ ]:
def collator(batch):
  out =  {
      'input_ids': torch.stack([(x['input_ids'][0]) for x in batch]),
      'attention_mask': torch.stack([x['attention_mask'][0] for x in batch]),
      'labels': torch.stack([x['labels'].clone().detach() for x in batch])
  }
  return out

In [ ]:
import torch.nn.functional as F

class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight, gamma, reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, ignore_index=-100, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [ ]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_labels=7):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs

        weights = torch.tensor([0.85, 1.55, 1.55, 1.65, 1.65, 1.85, 1.85]).cuda()  # The no-class label has too many examples, we need to weight the loss - this probably needs further tuning 
        gamma=5
        loss_fct = FocalLoss(weight=weights, gamma=gamma)
        loss = loss_fct(logits.view(-1, num_labels), labels.long().view(-1))
        return (loss, outputs) if return_outputs else loss

    def evaluation_loop(self, dataloader, description, prediction_loss_only=None, ignore_keys=None, metric_key_prefix="eval", num_labels=4):
      args = self.args
      prediction_loss_only = prediction_loss_only if prediction_loss_only is not None else args.prediction_loss_only

      self.model.eval()

      all_losses = []
      all_preds = []
      all_labels = []
      for step, sample in enumerate(dataloader):
        for i in range(0, len(sample['labels'])):
          inputs = {}
          inputs['input_ids'] = torch.stack([sample['input_ids'][i].cuda()])
          inputs['attention_mask'] = torch.stack([sample['attention_mask'][i].cuda()])
          inputs['labels'] = torch.stack([sample['labels'][i].cuda()])
          labels = inputs['labels'][0].cpu().numpy()
          
          (loss, logits) = self.compute_loss(self.model, inputs, return_outputs=True)
          logits = logits[0].cpu().detach().numpy()
          preds = np.argmax(nn.Softmax(dim=-1)(torch.tensor(logits)).numpy(), axis=-1)

          all_losses = np.concatenate((all_losses, [loss.detach().cpu().numpy()]), axis=0)

          preds = preds[labels != -100]
          labels = labels[labels != -100]
          all_preds = np.concatenate((all_preds, preds))
          all_labels = np.concatenate((all_labels, labels))

      metrics = {}
      metrics['macro_f1'] = f1_score(all_labels, all_preds, average='macro')
      metrics['macro_precision'] = precision_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['macro_recall'] = recall_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['micro_f1'] = f1_score(all_labels, all_preds, average='micro')
      metrics['micro_precision'] = precision_score(all_labels, all_preds, average='micro', zero_division=0)
      metrics['micro_recall'] = recall_score(all_labels, all_preds, average='micro', zero_division=0)

      metrics['macro_f1_no_o'] = f1_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6])
      metrics['macro_precision_no_o'] = precision_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['macro_recall_no_o'] = recall_score(all_labels, all_preds, average='macro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['micro_f1_no_o'] = f1_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6])
      metrics['micro_precision_no_o'] = precision_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)
      metrics['micro_recall_no_o'] = recall_score(all_labels, all_preds, average='micro', labels=[1, 2, 3, 4, 5, 6], zero_division=0)

      for key in list(metrics.keys()):
        if not key.startswith(metric_key_prefix):
          metrics[metric_key_prefix + '_' + key] = metrics.pop(key)
      
      metrics[metric_key_prefix + '_loss'] = all_losses.mean().item()

      return EvalLoopOutput(predictions=all_preds, label_ids=all_labels, metrics=metrics, num_samples=len(dataloader))

In [ ]:
def my_hp_space_ray(trial):
    from ray import tune

    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "num_train_epochs": tune.choice(range(8, 20)),
        "weight_decay": tune.uniform(0.0, 0.5),
        "per_device_train_batch_size": tune.choice([4, 8, 16]),  #<16 definetly not working
    }

# Hyperparameter Tuning

In [ ]:
def model_init():
    x = NerModel(BertEmbModel)
    x.sci_embeddings.requires_grad = False
    return x

trainer = MultilabelTrainer(model_init=model_init,
                            args=training_args,
                            train_dataset=local_hp_train,
                            eval_dataset=local_hp_val,
                            data_collator=collator  # defines how to merge data into batches, using the collator function above
                            )

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

best_run = trainer.hyperparameter_search(backend="ray", 
                                         resources_per_trial={"gpu": 1, "cpu": 0},
                                         n_trials=16, 
                                         direction="maximize", 
                                         hp_space=my_hp_space_ray,
                                         search_alg=HyperOptSearch(metric='eval_micro_recall_no_o', mode="max"),  #'eval_*f1_micro'
                                         scheduler=ASHAScheduler(metric='eval_micro_recall_no_o', mode="max")
                                         )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
best_run

BestRun(run_id='c691df3e', objective=5.921519281375328, hyperparameters={'learning_rate': 2.0067205312195233e-05, 'num_train_epochs': 18, 'weight_decay': 0.24799733516738603, 'per_device_train_batch_size': 4})

# Cossvalidation

In [ ]:
#import tensorflow as tf
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from torch import nn
from transformers import Trainer

#print('GPU detected:', tf.config.list_physical_devices('GPU'))
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
results = np.zeros(5)
resultss = np.zeros(5)
dataset = ConcatDataset([train, test])



In [ ]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    #train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    #test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    local_train = []
    #i = 0
    for idx in train_ids:
      #if i <= 1920:
        local_train.append(dataset[idx])
      #i += 1
    
    local_test = []
    #i = 0
    for idx in test_ids:
     # if i <= 480:
        local_test.append(dataset[idx])
      #i += 1

    training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=2.0067205312195233e-05, 
                                      weight_decay=0.24799733516738603,
                                      num_train_epochs=18, 
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=4, 
                                      per_device_train_batch_size=4,
                                      logging_steps=50,
                                      save_steps=len(local_train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      eval_steps=len(local_train) // batch_size
                                      )
    #learning_rate 2.0067205312195233e-05
    #num_train_epochs 18
    #weight_decay 0.24799733516738603
    #per_device_train_batch_size 4

    # Init the neural network
    ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
    
    trainer = MultilabelTrainer(model=ner_model,
                                args=training_args,
                                train_dataset=local_train,
                                eval_dataset=local_test,
                                data_collator=collator  # defines how to merge data into batches, using the collator function above
                                )

    #Loading Best parameters
    #load_param()

    #Training
    trainer.train()
          
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(ner_model.state_dict(), save_path)

    # Evaluationfor this fold
    #correct, total = 0, 0
    with torch.no_grad():
      result = trainer.evaluate(local_test)
      print(result)

      # Print accuracy
      print('Accuracy for fold ', fold, ': ', result['eval_micro_f1_no_o'], ' -- ', result['eval_micro_f1'])
      print('--------------------------------')
      results[fold] = result['eval_micro_f1_no_o']
      resultss[fold] = result['eval_micro_f1']
      del result
    


FOLD 0
--------------------------------


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1606
  Num Epochs = 18
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 7218


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
401,0.055800,0.094043,0.225512,0.204765,0.283355,0.865782,0.865782,0.865782,0.104192,0.082090,0.169515,0.280429,0.308009,0.257382
802,0.021000,0.063082,0.412475,0.432455,0.456696,0.889804,0.889804,0.889804,0.321172,0.344004,0.373238,0.472323,0.464133,0.480807
1203,0.013600,0.053966,0.459713,0.499144,0.495224,0.887640,0.887640,0.887640,0.376487,0.420387,0.419966,0.491893,0.458884,0.530020
1604,0.008200,0.040598,0.574147,0.537785,0.642506,0.901816,0.901816,0.901816,0.510323,0.464511,0.593326,0.610296,0.549941,0.685531
2005,0.005000,0.040516,0.630098,0.685535,0.681431,0.914665,0.914665,0.914665,0.575049,0.637449,0.637152,0.663034,0.615514,0.718504
2406,0.002600,0.046042,0.716589,0.716774,0.739156,0.920461,0.920461,0.920461,0.675780,0.674528,0.703549,0.687515,0.653795,0.724902
2807,0.002500,0.045439,0.741102,0.744778,0.743408,0.928701,0.928701,0.928701,0.703833,0.707746,0.706895,0.714390,0.704643,0.724409
3208,0.001500,0.041846,0.744103,0.738045,0.765022,0.923883,0.923883,0.923883,0.707845,0.698766,0.734214,0.713860,0.667810,0.766732
3609,0.000800,0.042301,0.752162,0.737203,0.774768,0.924791,0.924791,0.924791,0.717263,0.698113,0.745299,0.717759,0.677744,0.762795
4010,0.001400,0.053005,0.742845,0.756787,0.743160,0.929050,0.929050,0.929050,0.705860,0.721693,0.706660,0.717191,0.705910,0.728839


Saving model checkpoint to trained_scibert_ner_model/checkpoint-401
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-802
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1203
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1604
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2005
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2406
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2807
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.7564090883256013, 'eval_macro_precision': 0.7608361258145833, 'eval_macro_recall': 0.7591901593098086, 'eval_micro_f1': 0.9310754189944134, 'eval_micro_precision': 0.9310754189944134, 'eval_micro_recall': 0.9310754189944134, 'eval_macro_f1_no_o': 0.7215469613754554, 'eval_macro_precision_no_o': 0.7265477979294035, 'eval_macro_recall_no_o': 0.7249552509656101, 'eval_micro_f1_no_o': 0.7238933724627049, 'eval_micro_precision_no_o': 0.7194944093339816, 'eval_micro_recall_no_o': 0.7283464566929134, 'eval_loss': 0.058522603419891614, 'eval_runtime': 5.062, 'eval_samples_per_second': 19.755, 'eval_steps_per_second': 4.939, 'epoch': 18.0}
Accuracy for fold  0 :  0.7238933724627049  --  0.9310754189944134
--------------------------------
FOLD 1
--------------------------------


...

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9840289037510175, 'eval_macro_precision': 0.97511306033152, 'eval_macro_recall': 0.9932961650838309, 'eval_micro_f1': 0.9957391244588001, 'eval_micro_precision': 0.9957391244588001, 'eval_micro_recall': 0.9957391244588001, 'eval_macro_f1_no_o': 0.9817581506388976, 'eval_macro_precision_no_o': 0.9711112844790223, 'eval_macro_recall_no_o': 0.9928142842048985, 'eval_micro_f1_no_o': 0.9835817125536752, 'eval_micro_precision_no_o': 0.9744744744744744, 'eval_micro_recall_no_o': 0.9928607853136155, 'eval_loss': 0.0009351297551529569, 'eval_runtime': 5.0313, 'eval_samples_per_second': 19.876, 'eval_steps_per_second': 4.969, 'epoch': 18.0}
Accuracy for fold  3 :  0.9835817125536752  --  0.9957391244588001
--------------------------------
FOLD 4
--------------------------------


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1607
  Num Epochs = 18
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 7218


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
401,0.030000,0.032422,0.983030,0.982142,0.984094,0.995726,0.995726,0.995726,0.980501,0.979285,0.981921,0.980035,0.973394,0.986768
802,0.009400,0.012264,0.987466,0.986102,0.988938,0.995726,0.995726,0.995726,0.985769,0.983999,0.987665,0.983573,0.976908,0.990331
1203,0.005700,0.005665,0.986691,0.985022,0.988448,0.995933,0.995933,0.995933,0.984839,0.982752,0.987027,0.984055,0.978852,0.989313
1604,0.004400,0.003766,0.984756,0.979936,0.989827,0.994761,0.994761,0.994761,0.982694,0.976739,0.988942,0.979899,0.967742,0.992366
2005,0.001900,0.002973,0.984621,0.980860,0.988658,0.995450,0.995450,0.995450,0.982464,0.977804,0.987445,0.982120,0.972084,0.992366
2406,0.001600,0.001747,0.988779,0.987822,0.989843,0.996760,0.996760,0.996760,0.987194,0.985926,0.988589,0.987098,0.981388,0.992875
2807,0.001200,0.001371,0.989131,0.987824,0.990523,0.996829,0.996829,0.996829,0.987605,0.985928,0.989383,0.987604,0.981891,0.993384
3208,0.001000,0.001559,0.986740,0.983952,0.989659,0.995795,0.995795,0.995795,0.984909,0.981423,0.988547,0.983607,0.975000,0.992366
3609,0.000600,0.000948,0.990241,0.988989,0.991559,0.997036,0.997036,0.997036,0.988881,0.987287,0.990551,0.988354,0.983375,0.993384
4010,0.000700,0.000890,0.987853,0.985356,0.990455,0.996415,0.996415,0.996415,0.986154,0.983048,0.989383,0.986108,0.978937,0.993384


Saving model checkpoint to trained_scibert_ner_model/checkpoint-401
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-802
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1203
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1604
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2005
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2406
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2807
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9895794222891139, 'eval_macro_precision': 0.987870806181, 'eval_macro_recall': 0.9913473548142961, 'eval_micro_f1': 0.9968288983868744, 'eval_micro_precision': 0.9968288983868744, 'eval_micro_recall': 0.9968288983868744, 'eval_macro_f1_no_o': 0.9881286282398439, 'eval_macro_precision_no_o': 0.9859957062527002, 'eval_macro_recall_no_o': 0.9903305190585892, 'eval_micro_f1_no_o': 0.9875980764363452, 'eval_micro_precision_no_o': 0.9823766364551864, 'eval_micro_recall_no_o': 0.9928753180661578, 'eval_loss': 0.0005622432717973425, 'eval_runtime': 5.0315, 'eval_samples_per_second': 19.875, 'eval_steps_per_second': 4.969, 'epoch': 18.0}
Accuracy for fold  4 :  0.9875980764363452  --  0.9968288983868744
--------------------------------


# Results

In [ ]:
results

array([0.72389337, 0.93099469, 0.95901209, 0.98358171, 0.98759808])

In [ ]:
resultss

array([0.93107542, 0.98216308, 0.99074567, 0.99573912, 0.9968289 ])

In [ ]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
key = 0
for value in results:
  print(f'Fold {key}: {value} %')
  sum += value
  key += 1
print(f'Average: {sum/len(results)} %')

K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.7238933724627049 %
Fold 1: 0.9309946918993769 %
Fold 2: 0.9590120861797161 %
Fold 3: 0.9835817125536752 %
Fold 4: 0.9875980764363452 %
Average: 0.9170159879063636 %


# Pytorch Training - Loop UPDT

In [ ]:
device = 'cuda'

In [ ]:
training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=2.0067205312195233e-05, 
                                      num_train_epochs=18, 
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=4, 
                                      per_device_train_batch_size=4,
                                      logging_steps=50,
                                      save_steps=len(train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      weight_decay=0.24799733516738603,
                                      eval_steps=len(train) // batch_size,
                                      report_to='all'
                                      )

#learning_rate 2.0067205312195233e-05
#num_train_epochs 18
#weight_decay 0.24799733516738603
#per_device_train_batch_size 4

#load_param()

In [ ]:
loop_val = 10
loop_results = np.zeros(loop_val)
loop_resultss = np.zeros(loop_val)
for r in range(loop_val):

  ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training

  trainer = MultilabelTrainer(
      model=ner_model, 
      args=training_args, 
      train_dataset=train, 
      eval_dataset=test,
      data_collator=collator  # defines how to merge data into batches, using the collator function above
  )

  # Print
  print(f'Train run #{r}')
  print('--------------------------------')

  trainer.train()

  # Process is complete.
  print('Training process has finished.')

  # Print about testing
  print('Starting testing')

  with torch.no_grad():
    result = trainer.evaluate(test)
    print(result)

    # Print accuracy
    print('Accuracy for fold ', r, ': ', result['eval_micro_f1_no_o'], ' -- ', result['eval_micro_f1'])
    print('--------------------------------')
    loop_results[r] = result['eval_micro_f1_no_o']
    loop_resultss[r] = result['eval_micro_f1']
    del result

  if r > 0:
    if loop_results[r] < loop_results[r-1]:
      save_path = f'./model-fold-{r}.pth'
      torch.save(ner_model.state_dict(), save_path)

  print('Testing process has finished.')

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1522
  Num Epochs = 18
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6840


Train run #0
--------------------------------


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
380,0.077100,0.118436,0.177925,0.158943,0.229566,0.851687,0.851687,0.851687,0.049659,0.032567,0.104508,0.147969,0.195402,0.119066
760,0.041500,0.069224,0.222765,0.245620,0.245851,0.865596,0.865596,0.865596,0.103458,0.138006,0.121623,0.217516,0.411255,0.147860
1140,0.014500,0.060430,0.383633,0.392691,0.454025,0.880897,0.880897,0.880897,0.287732,0.298431,0.369726,0.431978,0.434014,0.429961
1520,0.014000,0.051201,0.386408,0.393632,0.477451,0.877652,0.877652,0.877652,0.290846,0.297719,0.398589,0.436900,0.415439,0.460700
1900,0.008300,0.046760,0.498633,0.543901,0.552000,0.889707,0.889707,0.889707,0.421724,0.472824,0.485664,0.514317,0.486796,0.545136
2280,0.004800,0.043132,0.606463,0.658266,0.672886,0.895676,0.895676,0.895676,0.547886,0.605828,0.627799,0.573321,0.531012,0.622957
2660,0.002700,0.043045,0.663321,0.673452,0.707205,0.907384,0.907384,0.907384,0.613404,0.623591,0.666204,0.610712,0.579462,0.645525
3040,0.001700,0.045243,0.669025,0.663476,0.722137,0.906804,0.906804,0.906804,0.620266,0.611841,0.684135,0.617234,0.580336,0.659144
3420,0.002600,0.043919,0.732891,0.715239,0.760199,0.916135,0.916135,0.916135,0.694904,0.672645,0.728393,0.677330,0.642036,0.716732
3800,0.001400,0.048893,0.750041,0.737443,0.771126,0.922511,0.922511,0.922511,0.714359,0.698518,0.740085,0.695473,0.669669,0.723346


Saving model checkpoint to trained_scibert_ner_model/checkpoint-380
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-760
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1140
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1520
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1900
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2280
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2660
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished.
Starting testing


...

Training process has finished.
Starting testing


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1522
  Num Epochs = 18
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 6840


{'eval_macro_f1': 0.7361031218931592, 'eval_macro_precision': 0.7501761339781974, 'eval_macro_recall': 0.7249939261569651, 'eval_micro_f1': 0.9228005100266605, 'eval_micro_precision': 0.9228005100266605, 'eval_micro_recall': 0.9228005100266605, 'eval_macro_f1_no_o': 0.6982204303103279, 'eval_macro_precision_no_o': 0.7146553432532681, 'eval_macro_recall_no_o': 0.6852432997573564, 'eval_micro_f1_no_o': 0.6906754915320226, 'eval_micro_precision_no_o': 0.691079080638878, 'eval_micro_recall_no_o': 0.690272373540856, 'eval_loss': 0.06980091975617315, 'eval_runtime': 10.4734, 'eval_samples_per_second': 11.553, 'eval_steps_per_second': 2.96, 'epoch': 18.0}
Accuracy for fold  38 :  0.6906754915320226  --  0.9228005100266605
--------------------------------
Testing process has finished.
Train run #39
--------------------------------


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
380,0.024700,0.104479,0.732387,0.745551,0.721228,0.923264,0.923264,0.923264,0.693920,0.709848,0.680327,0.689573,0.704260,0.675486
760,0.008900,0.089304,0.725969,0.799297,0.671474,0.923728,0.923728,0.923728,0.686693,0.774594,0.620692,0.683135,0.754468,0.624125
1140,0.004000,0.072586,0.714248,0.697758,0.738636,0.910282,0.910282,0.910282,0.673737,0.652449,0.704189,0.662672,0.621171,0.710117
1520,0.002500,0.068425,0.727229,0.739098,0.724047,0.916483,0.916483,0.916483,0.688447,0.701195,0.685818,0.679153,0.654749,0.705447
1900,0.001900,0.070226,0.721357,0.746753,0.706585,0.920366,0.920366,0.920366,0.681384,0.710794,0.664368,0.689189,0.683908,0.694553
2280,0.001300,0.068931,0.745837,0.764205,0.732955,0.922801,0.922801,0.922801,0.709840,0.731660,0.694418,0.696133,0.706165,0.686381
2660,0.001000,0.067664,0.741608,0.760482,0.727215,0.923380,0.923380,0.923380,0.704767,0.727264,0.687494,0.694378,0.706688,0.682490
3040,0.000600,0.059130,0.740140,0.729284,0.755924,0.916657,0.916657,0.916657,0.703632,0.689537,0.723450,0.687292,0.655941,0.721790


Saving model checkpoint to trained_scibert_ner_model/checkpoint-380
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-760
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1140
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1520
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1900
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2280
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2660
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

In [ ]:
loop_results

In [ ]:
loop_resultss

In [ ]:
sum = 0.0
for value in loop_results:
  sum += value
print(f'Average micro_f1_no_o: {sum/len(loop_results)} %')

sum = 0.0
for value in loop_resultss:
  sum += value
print(f'Average micro_f1: {sum/len(loop_results)} %')

In [ ]:
ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
ner_model.load_state_dict(torch.load(save_path))
ner_model.eval()

# Get Values of Thruth

In [ ]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()
ner_model.to('cuda')

In [ ]:
output_preds = []
output_real = []
for x in range(len(test)):
  inputs1 = test[x]['input_ids'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs2 = test[x]['attention_mask'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs = {'input_ids': inputs1,  'attention_mask': inputs2}
  #print(inputs)

  temp_test = test[x]
  temp_out = temp_test.pop("labels")
  output_real.append(np.array(temp_out[temp_out != -100])) 

  gen_preds = ner_model(**inputs)
  label_preds = np.argmax(gen_preds.cpu().detach().numpy(), axis=-1)[0]
  output_preds.append(label_preds[temp_out != -100])


In [ ]:
for x in range(len(output_real)):
  output_real[x] = [ID2Entity(y) for y in output_real[x]]
  output_preds[x] = [ID2Entity(z) for z in output_preds[x]]

In [ ]:
#output_real[28]

In [ ]:
from sklearn.metrics import mean_squared_error, multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 9]
plt.rcParams["figure.autolayout"] = True
plt.rcParams.update({'font.size': 13})

labels = ["None", 'B-TASK', 'I-TASK', 'B-DATASET', 'I-DATASET', 'B-METRIC', 'I-METRIC']

cm = confusion_matrix(output_real[0], output_preds[0], labels=labels)
for x in range(len(output_real)-1):
  cm += confusion_matrix(output_real[x+1], output_preds[x+1], labels=labels)
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)

In [ ]:
del cm

In [ ]:
len(output_real)

# Test over Text

In [ ]:
def prepare_input(txt):
  inputs = BertTokenizer(txt, return_tensors='pt', padding='max_length', truncation=True, max_length=150).to('cuda')
  return inputs

input_text = ["English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement."]
##input_text = ["The transient analysis of gyro-elastic structured media, composed of periodically placed masses interconnected by elastic rods and attached to gyroscopic spinners, is presented."] 
##input_text = ["The results indicated that thermal curing promoted the early strength of mortars, while decreased the late strength of mortars."] 
##input_text = ["A wide variety of processes are attested in the literature, and we find different forms of clippings in our data, including mixtures of different clippings, homophone respellings, phonetic respellings in-cluding informal oral forms, initialisms (but no acronyms), and mixtures of clipping together with homo-phone and phonetic respellings."] 

#input_text = ["The goal is to accurately predict the running time of applications for task scheduling and job migration."] 
#input_text = ["This paper reports on the development of a cross-domain framework for describing complex design practices."] 
#input_text = ["Studies of inequality in China typically ignore cost of liv-ing differences between areas."] 
#input_text = ["The present study was designed to explore the long-term differences be-tween three mouse models for depression."]
#input_text = ["Finally, regarding professional competencies, teachers appeared to be largely unprepared to conduct language assessments consistent with the LAR demands."] 

##input_text = ["propose a fast and reliable restoration method of virtual resources on OpenStack when physical servers or virtual machines are down."] 
##input_text = ["The results from our simulations reveal that the network assisted adaptation clearly outperforms the purely client-based DASH heuristics in some of the metrics, not all of them, particularly, in situations when the achievable throughput is moderately high or the link quality of the mobile clients does not differ from each other substantially."] 
##input_text = ["For hard rock drilling in coal mine, the drilling efficiency and service life of polycrystalline diamond compact bit are very low."] 
##input_text = ["Capturing changes in foreign reserves and exchange rates through the exchange market pressure, this article investigates whether economic policy uncertainty plays any role in exchange market pressure movements while controlling for the effects of domestic and external factors."] 
##input_text = ["This paper presents design of an self contained actuators unit in wide area damping control of power system in stabilizing system response for both nominal system condition and during actuator faults."] 

##input_text = ["Ultrasound-based brain stimulation techniques may become a powerful new technique to modulate the human brain in a focal and targeted manner."] 
#input_text = ["Recent work pre-training Transformers with self-supervised objectives on large text corpora has shown great success when fine-tuned on downstream NLP tasks including text summarization."]

# Tokenize + pad
inputs = prepare_input(input_text)

#inputs
#print(inputs)

In [ ]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()

# Get predictions
preds = ner_model(**inputs).cpu().detach().numpy()
preds = np.argmax(preds, axis=-1)[0]
pred_labels = [ID2Entity(x) for x in preds]

# Convert token ids to text
tokens = BertTokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Display result
for token, label in zip(tokens, pred_labels):
  if token == '[SEP]':
    break
  if token == '[CLS]':
    continue
  print('{} -> {}'.format(token, label))

english -> None
is -> None
shown -> None
to -> None
be -> None
trans -> None
- -> None
context -> None
- -> None
free -> None
on -> None
the -> None
basis -> None
of -> None
coordination -> None
##s -> None
of -> None
the -> None
respectively -> None
type -> None
that -> None
involve -> None
strictly -> None
syntactic -> None
cross -> None
- -> None
serial -> None
agreement -> None
. -> None


# Model Save and Load

In [ ]:
print("Our model: \n\n", ner_model, '\n')
print("The state dict keys: \n\n", ner_model.state_dict().keys())

Our model: 

 NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
torch.save(ner_model.state_dict(), 'trained_model_dic.pth')

In [ ]:
# download checkpoint file
files.download('trained_model_dic.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

checkpoint.pth	  model-fold-1.pth  model-fold-4.pth  trained_model_dic.pth
dev.json	  model-fold-2.pth  sample_data       trained_scibert_ner_model
model-fold-0.pth  model-fold-3.pth  test.json	      train.json


Loading the model

In [ ]:
!ls

sample_data  test_500_v2.conll	train_1500_v2.conll  trained_model_dic.pth


In [ ]:
state_dict = torch.load('trained_model_dic.pth')
print(state_dict.keys())

odict_keys(['sci_embeddings.embeddings.position_ids', 'sci_embeddings.embeddings.word_embeddings.weight', 'sci_embeddings.embeddings.position_embeddings.weight', 'sci_embeddings.embeddings.token_type_embeddings.weight', 'sci_embeddings.embeddings.LayerNorm.weight', 'sci_embeddings.embeddings.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.attention.self.query.weight', 'sci_embeddings.encoder.layer.0.attention.self.query.bias', 'sci_embeddings.encoder.layer.0.attention.self.key.weight', 'sci_embeddings.encoder.layer.0.attention.self.key.bias', 'sci_embeddings.encoder.layer.0.attention.self.value.weight', 'sci_embeddings.encoder.layer.0.attention.self.value.bias', 'sci_embeddings.encoder.layer.0.attention.output.dense.weight', 'sci_embeddings.encoder.layer.0.attention.output.dense.bias', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.weight', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.intermediate.dense.weight', 'sci_emb

In [ ]:
ner_model = NerModel(BertEmbModel).to('cuda')
ner_model.load_state_dict(state_dict)

<All keys matched successfully>

# Obtain datasets' weights values (Do not run - fixed values)

In [ ]:
zero = 0
one=0
two=0 
three=0
four=0
five=0
six=0
local_set = train
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1
    
print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

0:  48170 1:  4364 2:  1859 3:  1133 4:  0 5:  0 6:  0


In [ ]:
zero = 0
one=0
two=0 
three=0
four=0
five=0
six=0
local_set = test
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1
    
print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

0:  14719 1:  1337 2:  854 3:  386 4:  0 5:  0 6:  0
